<a href="https://colab.research.google.com/github/OdysseusPolymetis/enssib_class/blob/main/2_nlp_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**NLP : tokenizing, lemmatizing, postagging**

---



## What's NLP and why do we need it ?
Generally speaking, loads of people use NLP as a preprocessing phase, for further textual treatment. And it is absolutely necessary if you want to avoid noise in statistical analysis and machine learning uses. Basically, preprocessing is **tokenizing**, **lemmatizing** and **postagging**.

## How do you do that without programming ?
I'll show you some basic tools you can easily use without knowing programming in python.

##**Some useful online tools**

###**UDPipe**
You'll find it [here](https://lindat.mff.cuni.cz/services/udpipe/).
<br>You can use that for short texts.
<br>One of its peculiarities is SVG dependency tree building.

###**Deucalion**
You'll find it [here](https://dh.chartes.psl.eu/deucalion/).
<br>Much more accurate for longer texts, not an easy output (although ready-to-use format). It's really good on Ancient French and Latin.

###**VoyantTools**
You'll find it [here](https://voyant-tools.org/).
<br>It's a visualization, directly online, but for more modules you can build it locally and it's really powerful and neat.

#**LE TAL : TOKENISATION, LEMMATISATION, POSTAGGING**

We're going to test **`stanza`**. There are loads of other modules on the matter (like`spacy` and `pie-extended`), but `stanza` is fairly easy to use and performs well on loads of languages.

In [ ]:
!pip install stanza

Let's try on a simple string first, which is encapsulated in the `catilinaires` variable.

In [ ]:
catilinaires="Quousque tandem abutere, Catilina, patientia nostra ? Quamdiu etiam furor iste tuus nos eludet ? Quem ad finem sese effrenata jactabit audacia ? Nihilne te nocturnum praesidium Palatii, nihil urbis vigiliae, nihil timor populi, nihil concursus bonorum omnium, nihil hic munitissimus habendi senatus locus, nihil horum ora vultusque moverunt ? Patere tua consilia non sentis ? Constrictam jam horum omnium scientia teneri conjurationem tuam non vides ? Quid proxima, quid superiore nocte egeris, ubi fueris, quos convocaveris, quid consilii ceperis, quem nostrum ignorare arbitraris ? O tempora ! O mores ! Senatus haec intellegit, consul videt. Hic tamen vivit."

##**stanza (précédemment Stanford CoreNLP)**

`stanza` has several language models at your disposal (here's a [list](https://stanfordnlp.github.io/stanza/performance.html)), which you can get using the basic language code, like `grc` for Ancient Greek or `la` for Latin. But you can also specify which model you want like below.

In [ ]:
import stanza
stanza.download('la', package="perseus")

We begin with building a Pipeline, to indicate which processors we want to use (no need to add `ner` if you don't need named entity recognition).

In [ ]:
nlp_stanza = stanza.Pipeline(lang='la', package="perseus", processors='tokenize,pos,lemma, depparse')

Now you can launch the nlp process. Here the `catilinaires_analyzed` is a stanza object, where all the informations you asked for in the Pipeline are encapsulated.

In [ ]:
catilinaires_analyzed=nlp_stanza(catilinaires)

In [ ]:
type(catilinaires_analyzed)

Here are some results, first for sentence division, and then for lemmatizing and postagging.

`stanza` usually divides the text in sentences, and each sentence is a list of tokens, which each contains attributes such as `.lemma`, or `.pos`.

In [ ]:
for sent in catilinaires_analyzed.sentences:
  print("XXXXX "+sent.text+" XXXXX")

In [ ]:
for sent in catilinaires_analyzed.sentences:
  for token in sent.words:
    print(token.text + ' - ' + token.lemma + ' - ' + token.pos)

# Let's try with a (much bigger) text.

As the text we're going to use is very long, we will use batch processing, that is, launching several processes at once for GPU management. You don't need to understand everything here, but basically we are going to launch several processes at once using the colab GPU.

The result will be the same as the one you'll get with a normal process, that is to say, in this case, a list (`words`), which contains each token that has been analyzed by `stanza`.

Once again, the following cell is just function declaration : it won't do anything unless you call it in your code later.

In [ ]:
def batch_process(text, nlp, batch_size=50):
    paragraphs = text.split('\n')
    batches = [paragraphs[i:i + batch_size] for i in range(0, len(paragraphs), batch_size)]

    words = []

    for batch in batches:
        batch_text = '\n'.join(batch)
        doc = nlp(batch_text)
        for sentence in doc.sentences:
            for word in sentence.words:
                token={}
                if word.lemma is not None:
                    token["word"]=word.text
                    token["lemma"]=word.lemma
                    token["pos"]=word.pos
                    words.append(token)

    return words

In [ ]:
import stanza
stanza.download('fr')
import string

Let's get the _Misérables_.

In [ ]:
!wget https://raw.githubusercontent.com/OdysseusPolymetis/enexdi2025_prep/refs/heads/main/miserables.txt

In [ ]:
filepath_of_text = "/content/miserables.txt"

In [ ]:
full_text = open(filepath_of_text, encoding="utf-8").read()

In [ ]:
nlp_stanza = stanza.Pipeline(lang='fr', processors='tokenize,mwt,pos,lemma')

This part may take some time (for me it took something like 4 minutes).

In [ ]:
miserables_analyzed = batch_process(full_text, nlp_stanza)

In [ ]:
print(miserables_analyzed[15:25])

For your own projects, you can get loads of stopword lists [here](https://github.com/stopwords-iso).


The upside of this approach (downloading a file) is that you just have to add potential useless words directly into the file you can see by clicking on the folder icon (📁) on the left side of the screen.

If you accidentally click (after having clicked on the folder icon) on the second folder icon (the open one followed by ".."), not to worry : the initial folder is in fact the one that is called "📁 `content`", which you can also open.

In [ ]:
!wget https://raw.githubusercontent.com/OdysseusPolymetis/enexdi2025_prep/refs/heads/main/stopwords_fr.txt

In [ ]:
stopwords = open("/content/stopwords_fr.txt",'r',encoding="utf8").read().split("\n")

In the following cell, we're going to get every stanza token, and distribute its values in three lists, `forms`, `lemmas` and `no_stop`. In the `forms` list, all the original words are going to be stored (that is to say, basically, the text itself). In the `lemmas` list, the lemmas of the entire text will be stored. Finally, in the `no_stop` list, we will store all the lemmas, except for stopwords and punctuation.

We will then use these lists to show why NLP and pre-processing is necessary to get readable information.

In [ ]:
forms = []
lemmas = []
no_stop = []

for token in miserables_analyzed:
    form = token["word"]
    lemma = token["lemma"]

    if lemma not in string.punctuation:
        forms.append(form)
        lemmas.append(lemma)

    if lemma not in string.punctuation and lemma not in stopwords:
        no_stop.append(lemma)

The two following cells will give you the number of words stored in the lists. It should be more than 0.

In [ ]:
len(lemmas)

In [ ]:
len(no_stop)

And now we're going to build some basic representations of our text to see why preprocessing is important.

You don't need to understand the whole code in the following cell. Just keep this in mind : it creates a word cloud - a visual representation where words from a list are displayed in different sizes based on how often they appear. More frequent words appear larger. In the end, it creates a circular visualization where the most important/frequent words stand out visually through their size, making it easy to quickly identify key themes.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

def create_word_cloud(words_list, title):
    text = ' '.join(words_list)

    radius = 495

    diameter = radius * 2
    center = radius
    x, y = np.ogrid[:diameter, :diameter]
    mask = (x - center) ** 2 + (y - center) ** 2 > radius ** 2
    mask = 255 * mask.astype(int)

    mask_rgba = np.dstack((mask, mask, mask, 255 - mask))

    wordcloud = WordCloud(repeat=False, width=diameter, height=diameter,
                          background_color=None, mode="RGBA", colormap='plasma',
                          mask=mask_rgba).generate(text)

    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

Remember, a function declaration doesn't do anything on its own : you need to call the function for it to work. The three following cells will create three clouds, one with the `forms` list, one with the `lemmas` list, and one with the `no_stop` list.

In [ ]:
create_word_cloud(forms, 'Word Cloud for Forms')

In [ ]:
create_word_cloud(lemmas, 'Word Cloud for Lemmas')

In [ ]:
create_word_cloud(no_stop, 'Word Cloud for Lemmas without stopwords')

If all went well, you'll be able to see that preprocessing is crucial to (visual, but also general) interpretation : the `forms` cloud is basically useless, while the `no_stop` cloud gives you much more information on your text.

In [ ]:
with open("/content/forms.txt", "w", encoding="utf8") as f, open("/content/lemmas.txt", "w", encoding="utf8") as f2, open("/content/pullito.txt", "w", encoding="utf8") as f3:
    f.write("\n".join(forms))
    f2.write("\n".join(lemmas))
    f3.write("\n".join(no_stop))